In [1]:
#Import the dependent libraries
from citipy import citipy
from datetime import datetime
import pandas as pd
import numpy as np
import timeit
import time
import requests

# Import the API key.
from config import weather_api_key

In [2]:
# Create a set of random latitude and longitude combinations

lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)
#Starting URL for weather Map API calls
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [4]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# Loop through all the cities in the list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
        time.sleep(5)

    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Add 1 to the record count.
    record_count += 1 
    
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        description = city_weather['weather'][0]['description']
        
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
       
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description":description })

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
    

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | grand river south east
City not found. Skipping...
Processing Record 2 of Set 1 | namatanai
Processing Record 3 of Set 1 | mar del plata
Processing Record 4 of Set 1 | leningradskiy
Processing Record 5 of Set 1 | punta arenas
Processing Record 6 of Set 1 | marcona
City not found. Skipping...
Processing Record 7 of Set 1 | port elizabeth
Processing Record 8 of Set 1 | port-gentil
Processing Record 9 of Set 1 | hervey bay
Processing Record 10 of Set 1 | atuona
Processing Record 11 of Set 1 | kodiak
Processing Record 12 of Set 1 | zhezkazgan
Processing Record 13 of Set 1 | cidreira
Processing Record 14 of Set 1 | albany
Processing Record 15 of Set 1 | kazalinsk
City not found. Skipping...
Processing Record 16 of Set 1 | biak
Processing Record 17 of Set 1 | saint george
Processing Record 18 of Set 1 | kapaa
Processing Record 19 of Set 1 | moron
Processing Record 20 of Set 1 | luderitz
Processing Reco

In [5]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Namatanai,PG,-3.6667,152.4333,84.63,69,55,5.46,broken clouds
1,Mar Del Plata,AR,-38.0023,-57.5575,72.09,90,72,13.00,broken clouds
2,Leningradskiy,RU,69.3833,178.4167,-28.30,97,98,6.31,overcast clouds
3,Punta Arenas,CL,-53.1500,-70.9167,50.11,53,75,26.46,broken clouds
4,Port Elizabeth,ZA,-33.9180,25.5701,62.89,88,0,5.75,clear sky
5,Port-Gentil,GA,-0.7193,8.7815,81.23,80,88,6.13,overcast clouds
6,Hervey Bay,AU,-25.2986,152.8535,84.04,67,2,8.01,clear sky
7,Atuona,PF,-9.8000,-139.0333,78.80,74,17,16.71,few clouds
8,Kodiak,US,57.7900,-152.4072,20.71,38,0,25.32,clear sky
9,Zhezkazgan,KZ,47.8043,67.7144,12.94,99,99,5.30,overcast clouds


In [7]:
# Create the output file (CSV).
output_data_file = "../Weather_Database/WeatherPy_Database.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")